In [1]:
from hmcmetad import hMCMetaD
import numpy as np
import hoomd
from hoomd import md
import coxeter

In [2]:
# Define shape
name = 'bipyramid'
n = 6 # n: number of center pyramid vertices
aspect = 1.12 # apsect: aspect ratio of the dipyramid
vertices = []

base_ngon = np.array(
    [[np.cos(i * 2 * np.pi / n),
    np.sin(i * 2 * np.pi / n), 0] for i in np.linspace(0, int(n-1), int(n))])
ngon = coxeter.shapes.ConvexPolygon(base_ngon)

vertices.extend(base_ngon)

h = ngon.circumcircle.radius * aspect
vertices.extend(np.array([[0, 0, h]]))
vertices.extend(np.array([[0, 0, -h]]))
shape = coxeter.shapes.ConvexPolyhedron(vertices)
shape.volume = 1.
verts = shape.vertices.tolist()
mass = 1
inertia = np.diagonal(shape.inertia_tensor) * mass
r_in = shape.maximal_centered_bounded_sphere.radius
r_circum = shape.minimal_centered_bounding_sphere.radius

In [3]:
# Define simulation parameters
final_pf = 0.36
run_params = dict(N_random = 2e5, N_compress = 2e5, N_eq = 2e7)
fname = f'{name}_pf_{final_pf}'

In [4]:
cpu = hoomd.device.CPU(notice_level=2)
seed = int(np.genfromtxt(f'./data/output_{fname}.txt', usecols=2)[-1])
sim = hoomd.Simulation(device=cpu, seed=seed)
sim.create_state_from_gsd(filename=f'./data/restart_{fname}.gsd')

kT = 1.0
dt = 0.0005
integrator = md.Integrator(dt=dt)
nl = md.nlist.Cell(buffer=0.4)

# Define ALJ parameters
sigma = 2.0 * r_in
r_cut = 2.0*r_circum + 0.15*sigma
alj = md.pair.aniso.ALJ(nl)
alj.r_cut[(name, name)] = r_cut
alj.params[(name, name)] = dict(epsilon=0.1,
                                sigma_i=sigma,
                                sigma_j=sigma,
                                alpha=0)

alj.shape[name] = dict(vertices=verts,
                       faces=shape.faces, rounding_radii=0)

integrator.forces.append(alj)

tau = 100*dt
bussi = md.methods.thermostats.Bussi(kT=kT, tau=tau)
nvt = md.methods.ConstantVolume(filter=hoomd.filter.All(), thermostat=bussi)

integrator.methods.append(nvt)
integrator.integrate_rotational_dof = True
sim.operations.integrator = integrator


sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=kT)
thermodynamic_properties = md.compute.ThermodynamicQuantities(
    filter=hoomd.filter.All())
sim.operations.computes.append(thermodynamic_properties)
sim.run(0)

In [7]:
log_period = 5e4
table_logger = hoomd.logging.Logger(categories=['scalar', 'string'])
table_logger.add(sim, quantities=['timestep', 'tps', 'seed', 'walltime'])
output_file = open(f'./data/output_{fname}.txt', mode='a', newline='\n')
table = hoomd.write.Table(output=output_file,
                          trigger=hoomd.trigger.Periodic(period=int(log_period)),
                          logger=table_logger)
sim.operations.writers.append(table)

logger = hoomd.logging.Logger()
logger.add(alj, quantities=['type_shapes', 'energy'])
logger.add(integrator, quantities=['linear_momentum'])
logger.add(thermodynamic_properties, quantities=['kinetic_temperature', 'pressure',
                                                 'kinetic_energy', 'translational_kinetic_energy',
                                                 'rotational_kinetic_energy', 'potential_energy',
                                                 'volume'])

In [8]:
########### Turn on WTmetaD ###########

cv_min = 0.0
cv_max = 0.5
bins = 30
init_height = 1.0
sigma = 0.01
gamma = 20
mc_stride = 100
metad_stride = mc_stride * 500
mode = 'steinhardt'

if mode == 'steinhardt':
    colvar_params = dict(r_max=2.0, l=6, average=False, wl=False, weighted=False, wl_normalize=False)

extra_colvar_mode = 'nematic'
extra_colvar_params = dict(director=[0, 0, 1])

hmcmetad_action = hMCMetaD(
    sim=sim,
    colvar_mode=mode,
    colvar_params=colvar_params,
    kT=kT,
    init_height=init_height,
    mc_stride=mc_stride,
    metad_stride=metad_stride,
    sigma=sigma,
    gamma=gamma,
    cv_min=cv_min,
    cv_max=cv_max,
    bins=bins,
    seed=seed,
    bias_mode=True,
    restart=True,
    restart_fn='./data/WTMetaD_history.txt',
    extra_colvar_mode=extra_colvar_mode,
    extra_colvar_params=extra_colvar_params,
    verbose=True
)

op = hoomd.update.CustomUpdater(action=hmcmetad_action, trigger=1)
sim.operations += op
sim.operations += hmcmetad_action.get_writer('./data/WTMetaD_log.hdf5')

Reading history ....
Len of initial cv history: 400
Len of initial height history: 400
last 5 element in initial cv array: [0.35514417 0.34981233 0.34818953 0.31037492 0.31431228]
last 5 element in inital height array: [0.30564048 0.31641391 0.31713213 0.52486047 0.47898268]



In [9]:
dump_period = metad_stride
traj_writer = hoomd.write.GSD(filename=f'./data/traj_{fname}.gsd',
                              trigger=hoomd.trigger.Periodic(int(dump_period), phase=1),
                              mode='ab',
                              logger=logger,
                              dynamic=['property', 'momentum'])
sim.operations.writers.append(traj_writer)

total_timestep = run_params['N_random'] + run_params['N_compress'] + run_params['N_eq']
while sim.timestep < total_timestep:
    step_size = int(total_timestep - sim.timestep)
    sim.run(min(1e4, step_size))
    hmcmetad_action.save_history('./data/WTMetaD_history.txt')

    for writer in sim.operations.writers:
        if hasattr(writer, "flush"):
            writer.flush()

hmcmetad_action.save_configuration(config_fn='./data/restart_{fname}.gsd', logger=logger)

Saving status before exiting ...
PE of the current configuration: 164.993742382301
PE of the current configuration: 164.993742382301
current bias pot: 9.162314600667015
current cv: 0.2967943847179413
last 5 element in cv array: [0.35514417 0.34981233 0.34818953 0.31037492 0.31431228]
last 5 element in height array: [0.30564048 0.31641391 0.31713213 0.52486047 0.47898268]
Len of cv array: 400
Len of height array: 400

